# Data Preprocessing
This notebook aims to preprocess our data for the usage of Multi-Armed Bandit algorithms

**Prepared by: Group 7**

*Chan Cheah Cha A0189006A, Chua Kai Bing A0185606Y, Goh Jia Yi A0185610J, Lim Jia Qi A0189626M, Tan Zen Wei A0188424X*

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
beer_df = pd.read_csv('/content/drive/MyDrive/BT4014/Codes/Data/beer_reviews.csv')
beer_df

,brewery_id,brewery_name,review_time,review_overall,review_aroma,review_appearance,review_profilename,beer_style,review_palate,review_taste,beer_name,beer_abv,beer_beerid
0,10325,Vecchio Birraio,1234817823,1.5,2.0,2.5,stcules,Hefeweizen,1.5,1.5,Sausa Weizen,5.0,47986
1,10325,Vecchio Birraio,1235915097,3.0,2.5,3.0,stcules,English Strong Ale,3.0,3.0,Red Moon,6.2,48213
2,10325,Vecchio Birraio,1235916604,3.0,2.5,3.0,stcules,Foreign / Export Stout,3.0,3.0,Black Horse Black Beer,6.5,48215
3,10325,Vecchio Birraio,1234725145,3.0,3.0,3.5,stcules,German Pilsener,2.5,3.0,Sausa Pils,5.0,47969
4,1075,Caldera Brewing Company,1293735206,4.0,4.5,4.0,johnmichaelsen,American Double / Imperial IPA,4.0,4.5,Cauldron DIPA,7.7,64883
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1586609,14359,The Defiant Brewing Company,1162684892,5.0,4.0,3.5,maddogruss,Pumpkin Ale,4.0,4.0,The Horseman's Ale,5.2,33061
1586610,14359,The Defiant Brewing Company,1161048566,4.0,5.0,2.5,yelterdow,Pumpkin Ale,2.0,4.0,The Horseman's Ale,5.2,33061
1586611,14359,The Defiant Brewing Company,1160702513,4.5,3.5,3.0,TongoRad,Pumpkin Ale,3.5,4.0,The Horseman's Ale,5.2,33061
1586612,14359,The Defiant Brewing Company,1160023044,4.0,4.5,4.5,dherling,Pumpkin Ale,4.5,4.5,The Horseman's Ale,5.2,33061


## Getting final shortlisted beers (Same steps as EDA)

In [ ]:
# Sort by highest review count
beer_df['count'] = 1
beer_count = beer_df[['beer_name','count']].groupby('beer_name').sum().sort_values(by=['count'],ascending=False)
beer_count = beer_count.reset_index()

In [ ]:
# Top 10 Most Popular Beer by Review Count
# We will exclude these beers from our recommendation list
mostpop10 = list(beer_count['beer_name'][:10])
mostpop10

['90 Minute IPA',
 'India Pale Ale',
 'Old Rasputin Russian Imperial Stout',
 'Sierra Nevada Celebration Ale',
 'Two Hearted Ale',
 'Stone Ruination IPA',
 'Arrogant Bastard Ale',
 'Sierra Nevada Pale Ale',
 'Stone IPA (India Pale Ale)',
 'Pliny The Elder']

In [ ]:
# Find out which beers have the highest review score
beer_reviews = beer_df[['beer_name','review_overall']].groupby('beer_name').mean().sort_values(by=['review_overall'],ascending=False)
beer_reviews = beer_reviews.reset_index()
beer_reviews.rename(columns={'review_overall': 'review_mean'}, inplace=True)  ##rename aggregated col

In [ ]:
# Join both df
# review_overall here is mean review score
beer_combined = pd.merge(beer_count, beer_reviews, on=["beer_name"])
beer_combined.head(30)

,beer_name,count,review_mean
0,90 Minute IPA,3290,4.145745
1,India Pale Ale,3130,3.842812
2,Old Rasputin Russian Imperial Stout,3111,4.174221
3,Sierra Nevada Celebration Ale,3000,4.168667
4,Two Hearted Ale,2728,4.329729
5,Stone Ruination IPA,2704,4.161612
6,Arrogant Bastard Ale,2704,4.078402
7,Sierra Nevada Pale Ale,2587,4.245845
8,Stone IPA (India Pale Ale),2575,4.257670
9,Pliny The Elder,2527,4.590028


In [ ]:
# Keep only Top 100 >= 4.0 review score and > 100 reviews
filtered = beer_combined[(beer_combined["count"] >=100) &(beer_combined["review_mean"]>= 4.0)].sort_values("review_mean",ascending=False)
filtered

,beer_name,count,review_mean
1415,Citra DIPA,252,4.630952
2164,Cantillon Blåbær Lambik,156,4.628205
670,Heady Topper,469,4.625800
2861,Deviation - Bottleworks 9th Anniversary,112,4.620536
122,Trappist Westvleteren 12,1272,4.617925
...,...,...,...
527,New Holland The Poet,561,4.000000
1771,Vichtenaar,198,4.000000
1744,St. Feuillien Cuvée De Noël,202,4.000000
729,Bière De Mars,441,4.000000


In [ ]:
# Only keep 100 beer that are not in the most popular 10 by review count
boolean_series = ~filtered['beer_name'].isin(mostpop10)
reco_df = filtered[boolean_series][:100]
reco_df

,beer_name,count,review_mean
1415,Citra DIPA,252,4.630952
2164,Cantillon Blåbær Lambik,156,4.628205
670,Heady Topper,469,4.625800
2861,Deviation - Bottleworks 9th Anniversary,112,4.620536
122,Trappist Westvleteren 12,1272,4.617925
...,...,...,...
238,Great Lakes Dortmunder Gold,896,4.295201
994,Wet,344,4.295058
1754,Ivan The Terrible,201,4.293532
270,Consecration,853,4.291911


## Transforming Data for MAB

In [ ]:
# Keep only reviews that mentioned the 100 beers we plan to recommend
reco_reviews = beer_df[beer_df['beer_name'].isin(recobeers)]
# Clean up reviews with NaN
reco_reviews = reco_reviews[reco_reviews['review_profilename'].notnull()].sort_values(by="review_profilename")
reco_reviews

,brewery_id,brewery_name,review_time,review_overall,review_aroma,review_appearance,review_profilename,beer_style,review_palate,review_taste,beer_name,beer_abv,beer_beerid,count
800247,18149,The Lost Abbey,1224438466,4.0,5.0,3.5,0110x011,American Wild Ale,4.5,5.0,Isabelle Proximus,7.00,42816,1
661342,3120,Alpine Beer Company,1243095767,4.5,4.5,5.0,0110x011,American Double / Imperial IPA,5.0,5.0,Exponential Hoppiness,11.00,12068,1
579247,2210,Firestone Walker Brewing Co.,1232206678,5.0,5.0,5.0,0110x011,American Strong Ale,5.0,5.0,Firestone 11 - Anniversary Ale,11.00,39873,1
33700,1199,Founders Brewing Company,1223914717,5.0,4.5,4.5,0110x011,American Double / Imperial Stout,5.0,5.0,Founders KBS (Kentucky Breakfast Stout),11.20,19960,1
614981,199,Ballast Point Brewing Company,1240193211,4.5,4.0,3.5,0110x011,American IPA,4.0,4.0,Sculpin India Pale Ale,7.00,29619,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155929,396,AleSmith Brewing Company,1166221983,4.5,5.0,4.0,zuggy9,American IPA,4.5,5.0,AleSmith IPA,7.25,3916,1
779267,13014,Surly Brewing Company,1293689525,4.0,4.0,4.5,zulufactor,American Double / Imperial IPA,4.0,5.0,Abrasive Ale,9.00,46849,1
422480,73,Great Lakes Brewing Company,1294898238,3.5,4.5,4.5,zulufactor,American Porter,4.0,4.5,Great Lakes Edmund Fitzgerald Porter,5.80,226,1
1491252,313,Brouwerij Westvleteren (Sint-Sixtusabdij van W...,1278358162,4.5,5.0,4.5,zwalk8,Quadrupel (Quad),5.0,5.0,Trappist Westvleteren 12,10.20,1545,1


In [ ]:
# Getting number of reviews made by each user
userreviews = reco_reviews[['review_overall','review_profilename']].groupby(by='review_profilename').agg('count').sort_values(by='review_overall',ascending=False)
userreviews

,review_overall
review_profilename,
dyan,114
mikesgroove,101
largadeer,100
Thorpe429,100
HopHead84,99
...,...
chiliboy98,1
chilibeer,1
chileman13,1


In [ ]:
# Create dictionary of users with their reviews
userdict = {} #initialise
temp = {}
user = None
# Loop through reviews
for n in range(len(reco_reviews)):
  # Check if user existing in dict
  if reco_reviews.iloc[n,6] != user:
    userdict[user] = temp 
    temp = {}
    user = reco_reviews.iloc[n,6]
  # Append review to user. If > 1 review, take average score
  if reco_reviews.iloc[n,10] in temp:
    counts = temp[reco_reviews.iloc[n,10]][1] + 1
    ratings = (temp[reco_reviews.iloc[n,10]][0] + reco_reviews.iloc[n,3])/counts
    temp[reco_reviews.iloc[n,10]] = [ratings,counts]
  else:
    temp[reco_reviews.iloc[n,10]] = [reco_reviews.iloc[n,3],1]
  userdict[user] = temp
  


In [ ]:
# Dictionary of users
userdict

{None: {},
 '0110x011': {'AleSmith IPA': [5.0, 1],
  'AleSmith Speedway Stout - Barrel Aged': [3.5, 1],
  'Alpha King Pale Ale': [5.0, 1],
  'Beatification': [5.0, 1],
  "Bell's Hopslam Ale": [4.5, 1],
  'Blind Pig IPA': [4.5, 1],
  'Cable Car': [4.0, 1],
  'Cantillon Saint Lamvinus': [5.0, 1],
  'Consecration': [4.5, 1],
  'Darkness': [3.5, 1],
  'Devil Over A Barrel - Bourbon Barrel-Aged Coffee Imperial Oatmeal Porter': [4.0,
   1],
  'Dreadnaught IPA': [4.5, 1],
  'Exponential Hoppiness': [4.5, 1],
  'Firestone 11 - Anniversary Ale': [5.0, 1],
  'Founders Breakfast Stout': [4.5, 1],
  'Founders KBS (Kentucky Breakfast Stout)': [5.0, 1],
  'Isabelle Proximus': [4.0, 1],
  'Pliny The Younger': [5.0, 1],
  'Pure Hoppiness': [5.0, 1],
  'Sanctification': [5.0, 1],
  'Sculpin India Pale Ale': [4.5, 1],
  'St. Bernardus Abt 12': [3.5, 1],
  'Supplication': [4.0, 1],
  'Temptation': [5.0, 1],
  'The Abyss': [4.5, 1],
  'Trappist Westvleteren 12': [4.5, 1],
  'Tröegs Nugget Nectar': [4.5, 1

In [ ]:
userdict.pop(None) # Remove initialising value
len(userdict)

10437

In [ ]:
# Create empty matrix
beer_matrix = pd.DataFrame(0, index=np.arange(len(userreviews)), columns=(recobeers))

In [ ]:
## Loop through dictionary and append values to matrix 
n = 0
for user,ratings in userdict.items(): #
  for beer in ratings.items(): #e.g ('Sierra', [4.5, 1])
    beer_matrix.loc[n,[beer[0]]] = beer[1][0]
  n = n + 1


## Final Matrix Used for MAB

In [ ]:
# Results
beer_matrix

,Citra DIPA,Cantillon Blåbær Lambik,Heady Topper,Deviation - Bottleworks 9th Anniversary,Trappist Westvleteren 12,Pliny The Younger,Founders CBS Imperial Stout,Live Oak HefeWeizen,Portsmouth Kate The Great,Rare Bourbon County Stout,Duck Duck Gooze,Reality Czeck,Weihenstephaner Hefeweissbier,Trappist Westvleteren 8,Zombie Dust,Masala Mama India Pale Ale,Vanilla Bean Aged Dark Lord,Kuhnhenn Bourbon Barrel Fourth Dementia,Cable Car,The Abyss,Unplugged Bohemian Lager,Sculpin India Pale Ale,Edward Pale Ale,Ephraim,Isabelle Proximus,Andechser Doppelbock Dunkel,Abner Imperial IPA,Supplication,Abrasive Ale,Czar Jack Imperial Stout,Parabola,Brute,Cantillon Lou Pepe - Gueuze,Silva Stout,Founders KBS (Kentucky Breakfast Stout),Gumballhead,Tröegs Nugget Nectar,White Rajah,Temptation,Bell's Black Note Stout,...,Trappistes Rochefort 10,Southampton Saison,"St. Bernardus Abt 12 (60th Anniversary ""Special Edition"")",Furious,Beatification,Exponential Hoppiness,Sanctification,Cantillon Cuvée Des Champions,Alpha King Pale Ale,Great Lakes Edmund Fitzgerald Porter,Dreadnaught IPA,The Event Horizon,Knuckle Sandwich,St. Bernardus Abt 12,Lunch,Schneider Weisse Weizenhell (Hefeweizen),Cantillon Lou Pepe - Kriek,Weihenstephaner Hefeweissbier Dunkel,Bourbon County Brand Coffee Stout,Barrel Aged Sexual Chocolate,Bitter Monk,Saison - Brett,Cuvee De Castleton,Devil Over A Barrel - Bourbon Barrel-Aged Coffee Imperial Oatmeal Porter,AleSmith IPA,Matt,Oak Aged Dark Lord Imperial Stout,Firestone 13 - Anniversary Ale,Bitter Brewer,Ayinger Celebrator Doppelbock,La Fin Du Monde,Moon Man,Darkness,Daisy Cutter Pale Ale,Cadillac Mountain Stout,Great Lakes Dortmunder Gold,Wet,Ivan The Terrible,Consecration,Hop Trip (Bond Street Series)
0,0.0,0.0,0.0,0.0,4.5,5.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,4.0,4.5,0.0,4.5,0.0,0.0,4.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,4.5,0.0,5.0,0.0,...,0.0,0.0,0.0,0.0,5.0,4.5,5.0,0.0,5.0,0.0,4.5,0.0,0.0,3.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.5,0.0,0.0,0.0,0.0,0.0,4.5,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,4.5,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.5,0.0,5.0,4.5,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.5,0.0,0.0,0.0,0.0,0.0,4.5,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,4.5,0.0,0.0,0.0,4.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10432,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.

In [ ]:
# Check if matrix is correct by checking how many values are there in a row
beer_matrix.iloc[:10].astype(bool).sum(axis=1)

0    28
1     1
2    17
3     1
4     1
5     5
6     1
7     1
8     1
9     2
dtype: int64

In [ ]:
# Cross check with length of user's dict
test = 0 
for user,ratings in userdict.items():
  if test < 10:
    print(len(ratings))
    test = test + 1

28
1
17
1
1
5
1
1
1
2


In [ ]:
# Save as csv
beer_matrix.to_csv ('/content/drive/MyDrive/BT4014/Codes/beer_matrix_100.csv', index = False, header=True)